# Interfacing Residue Analysis with INTERCAAT

In [10]:
!git clone https://gitlab.com/fiserlab.org/intercaat.git

Cloning into 'intercaat'...


In [1]:
import pandas as pd
import numpy as np
import os

In [48]:
## Load in experiment results
experiment_results_df = pd.read_csv('..\postprocessing\experiment_results.csv')#[0:1]
# experiment_results_df.head()

In [47]:
local_base_path = "C:\\Users\\Colby\\Documents\\GitHub\\SARS-CoV-2_N-Cytokine_Docking\\haddock\\postprocessing\\best_pdbs\\"

for index, row in experiment_results_df.iterrows():
    ## Create full path to best PDB file
    best_pdb_path = row['best_pdb_path'][1:]
    # pdb_path = f'{local_base_path}{os.path.basename(best_pdb_path)}'

    ## Run INTERCAAT and parse stdout
    intercaat_output = os.popen(f'python intercaat.py -pdb {os.path.basename(best_pdb_path)} -fp {local_base_path} -qc A -ic B ').read()
    intercaat_output_lines = intercaat_output.split('  Query Chain    |Interacting Chains| Dist | AtomClasses')[0].split('\n')
    intercaat_interfacing_residues_on_N_df = pd.DataFrame([x.split('    ') for x in intercaat_output_lines[1:]], columns=['Residue_number', 'Interactions'])
    intercaat_interfacing_residues_on_N_df.replace('', np.nan, inplace=True)
    intercaat_interfacing_residues_on_N_df.dropna(inplace=True)
    intercaat_interfacing_residues_on_N = ','.join(intercaat_interfacing_residues_on_N_df['Residue_number']).replace(' ', '')
    
    ## Add values to DataFrame
    experiment_results_df.loc[index, 'intercaat_interfacing_residues_on_N'] = intercaat_interfacing_residues_on_N

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "c:\ProgramData\PyMOL2.4.1\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "c:\ProgramData\PyMOL2.4.1\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [44]:
experiment_results_df_interface = experiment_results_df[['n_protein', 'cytokine_protein', 'experiment_name', 'intercaat_interfacing_residues_on_N']]

,n_protein,cytokine_protein,experiment_name,intercaat_interfacing_residues_on_N
0,MERS-CoV-N,CXCL13,MERS-CoV-N__CXCL13,"GLY28,ARG29,ASN30,PRO31,PRO33,ARG34,ILE85,ASN8..."


In [ ]:
## Write out results DataFrame
experiment_results_df_interface.to_csv("intercaat_interface_results.csv", index=False)